# Reactor atlas (GeoNuclearData)

High-signal plots of global reactor units with status filters, country choropleth, and site drilldowns.


In [ ]:
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go

TEMPLATE = "plotly_dark"
PALETTE = {
    "Operational": "#11b5e4",
    "Under construction": "#f19c65",
    "Planned": "#9b59b6",
    "Suspended": "#d94e5d",
    "Shutdown": "#6c757d",
}

# Resolve data root whether running from repo root or notebooks dir
cwd = Path.cwd().resolve()
if (cwd / "external" / "GeoNuclearData").exists():
    DATA_ROOT = cwd / "external" / "GeoNuclearData" / "data" / "csv" / "raw"
elif cwd.name == "notebooks" and (cwd.parent / "external" / "GeoNuclearData").exists():
    DATA_ROOT = cwd.parent / "external" / "GeoNuclearData" / "data" / "csv" / "raw"
else:
    raise RuntimeError("Cannot locate external/GeoNuclearData; run from repo root or notebooks directory.")

countries = pd.read_csv(DATA_ROOT / "1-countries.csv")
status_types = pd.read_csv(DATA_ROOT / "2-nuclear_power_plant_status_type.csv")
reactor_types = pd.read_csv(DATA_ROOT / "3-nuclear_reactor_type.csv")
plants = pd.read_csv(DATA_ROOT / "4-nuclear_power_plants.csv")

status_map = dict(zip(status_types.Id, status_types.Type))
type_map = dict(zip(reactor_types.Id, reactor_types.Type))
country_map = dict(zip(countries.Code, countries.Name))

units = plants.copy()
units["status"] = units["StatusId"].map(status_map)
units["reactor_type"] = units["ReactorTypeId"].map(type_map)
units["country"] = units["CountryCode"].map(country_map)
units["capacity_mw"] = units["Capacity"]
for col in ["ConstructionStartAt", "OperationalFrom", "OperationalTo", "LastUpdatedAt"]:
    if col in units.columns:
        units[col] = pd.to_datetime(units[col], errors="coerce")
units["operational_year"] = units["OperationalFrom"].dt.year
units["decade"] = (units["operational_year"] // 10 * 10).astype("Int64")

# Drop rows without coordinates or capacity
units_geo = units.dropna(subset=["Latitude", "Longitude", "capacity_mw"]).copy()

# Aggregate by site
site_agg = (
    units_geo.groupby(["Name", "CountryCode", "Latitude", "Longitude"], as_index=False)
    .agg(
        capacity_mw=("capacity_mw", "sum"),
        units=("Id", "count"),
        status=("status", lambda s: s.mode().iat[0] if not s.mode().empty else s.dropna().iloc[0] if not s.dropna().empty else None),
        reactor_types=("reactor_type", lambda s: ", ".join(sorted(set(s.dropna())))),
    )
)
site_agg["country"] = site_agg["CountryCode"].map(country_map)

# Country capacity
country_cap = units_geo.groupby("CountryCode", as_index=False).agg(
    capacity_mw=("capacity_mw", "sum"),
    units=("Id", "count"),
)
country_cap["country"] = country_cap["CountryCode"].map(country_map)

px.defaults.template = TEMPLATE

print("Units loaded:", len(units_geo))
print("Countries:", units_geo["CountryCode"].nunique())
print("Status mix:
", units_geo["status"].value_counts())


## Hero map (unit-level, status toggle)
- Bubble size = capacity (MW)
- Color = status (custom palette)
- Dropdown to toggle status subsets and show/hide planned/retired noise.


In [ ]:
# Prepare status subsets for an updatemenu
status_levels = ["Operational", "Under construction", "Planned", "Suspended", "Shutdown"]

buttons = []
for label in ["Operational", "Operational + UC", "All active (Operational+UC+Planned)", "All statuses"]:
    if label == "Operational":
        allowed = {"Operational"}
    elif label == "Operational + UC":
        allowed = {"Operational", "Under construction"}
    elif label == "All active (Operational+UC+Planned)":
        allowed = {"Operational", "Under construction", "Planned"}
    else:
        allowed = set(status_levels)
    mask = units_geo["status"].isin(allowed)
    subset = units_geo[mask]
    fig = px.scatter_geo(
        subset,
        lon="Longitude",
        lat="Latitude",
        size="capacity_mw",
        color="status",
        hover_name="Name",
        hover_data={
            "capacity_mw": True,
            "country": True,
            "reactor_type": True,
            "ReactorModel": True,
            "OperationalFrom": True,
            "OperationalTo": True,
        },
        size_max=18,
    )
    buttons.append({
        "label": label,
        "method": "update",
        "args": [
            {"lon": [subset["Longitude"]],
             "lat": [subset["Latitude"]],
             "marker": [{"size": subset["capacity_mw"], "sizemode": "area", "sizeref": 2 * subset["capacity_mw"].max() / (18 ** 2)}],
             "text": [subset["Name"]],
            },
            {"title": f"Reactor units – {label}"}
        ],
    })

# Default figure (Operational + UC)
default_mask = units_geo["status"].isin({"Operational", "Under construction"})
default_df = units_geo[default_mask]
fig = px.scatter_geo(
    default_df,
    lon="Longitude",
    lat="Latitude",
    size="capacity_mw",
    color="status",
    hover_name="Name",
    hover_data={
        "capacity_mw": True,
        "country": True,
        "reactor_type": True,
        "ReactorModel": True,
        "OperationalFrom": True,
        "OperationalTo": True,
    },
    size_max=18,
    title="Reactor units – Operational + Under construction",
)
fig.update_layout(legend_title_text="Status", updatemenus=[{
    "buttons": buttons,
    "direction": "down",
    "x": 0.02,
    "y": 0.95,
    "yanchor": "top"
}])
fig.show()


## Site density map (aggregated)
Aggregated by site; bubbles sized by total MW and colored by dominant status.


In [ ]:
fig = px.scatter_geo(
    site_agg,
    lon="Longitude",
    lat="Latitude",
    size="capacity_mw",
    color="status",
    hover_name="Name",
    hover_data={"capacity_mw": True, "units": True, "country": True, "reactor_types": True},
    size_max=22,
    title="Reactor sites (aggregated units)",
)
fig.update_layout(legend_title_text="Status")
fig.show()


## Capacity by country (choropleth)
Top-level view of where capacity sits.


In [ ]:
choropleth_df = country_cap[country_cap["capacity_mw"] > 0].copy()
fig = px.choropleth(
    choropleth_df,
    locations="CountryCode",
    color="capacity_mw",
    hover_name="country",
    color_continuous_scale="Blues",
    title="Nuclear capacity by country (MW)",
)
fig.update_geos(projection_type="natural earth")
fig.show()


## Timeline: operational starts by decade


In [ ]:
starts = units_geo.dropna(subset=["operational_year"]).copy()
starts["decade"] = (starts["operational_year"] // 10 * 10).astype(int)
by_decade = starts.groupby("decade", as_index=False).agg(units=("Id", "count"), capacity_mw=("capacity_mw", "sum"))
fig = px.bar(by_decade, x="decade", y="capacity_mw", text="units", title="Operational starts by decade (capacity added)")
fig.update_traces(textposition="outside")
fig.update_layout(xaxis_title="Decade", yaxis_title="Capacity added (MW)")
fig.show()


## Largest sites by capacity


In [ ]:
site_agg.sort_values("capacity_mw", ascending=False).head(20)


## Capacity by country (top 20)


In [ ]:
country_cap.sort_values("capacity_mw", ascending=False).head(20)
